# Cell Image augmentation
## Imports

In [1]:
import cv2
import numpy as np
import os
import albumentations as A

from config import *
from CellProcessor import read_yolo_labels

/Users/sapplab/Desktop/CellDataProcessing/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Augumentations

In [2]:
transform = A.Compose([
    #A.RandomCrop(width=256, height=256),
    A.RandomBrightnessContrast(p=0.2, brightness_limit=0.3, contrast_limit=0.15),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.4, label_fields=['class_labels']),
additional_targets={'phase_mask' : 'mask'})
#TODO add random scale

## Make dirs

In [3]:
AUGUMENTED_LABELED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Labels_phase_aug/"
AUGUMENTED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Phase_aug/"
AUGUMENTED_MASKS_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Masks_phase_aug/"

try:
    os.mkdir(AUGUMENTED_LABELED_IMAGES_DIR_PATH)
except OSError:
    pass

try:
    os.mkdir(AUGUMENTED_IMAGES_DIR_PATH)
except OSError:
    pass

try:
    os.mkdir(AUGUMENTED_MASKS_DIR_PATH)
except OSError:
    pass

## for every image annotation mask do augumentation

In [4]:
LABELED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Labeles_phase/"
IMAGES_MASKS_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Masks_phase/"
IMAGES_PHASE_PATH = IMAGES_PATH + CELL_TYPE + "_Crop_phase/"

images = sorted(os.listdir(IMAGES_PHASE_PATH))
len_images = len(images)
curr_image_num = 1

for image in images:
    print(str(curr_image_num) + "/" + str(len_images) + "-" + image)
    curr_image_num+=1
    base_image_path = IMAGES_PHASE_PATH + image
    annotation_path = LABELED_IMAGES_DIR_PATH + image.split(".")[0] + ".txt"
    mask_image_path = IMAGES_MASKS_DIR_PATH + image
    if os.path.exists(annotation_path):
        img_base = cv2.imread(base_image_path, cv2.IMREAD_UNCHANGED)
        img_mask = cv2.imread(mask_image_path, cv2.IMREAD_UNCHANGED)
        bboxes, class_labels = read_yolo_labels(annotation_path)

        transformed = transform(image=img_base, bboxes=bboxes, class_labels=class_labels, phase_mask=img_mask)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']
        transformed_mask = transformed['phase_mask']

        if len(transformed_bboxes):
            label_boxes = []
            image_file = AUGUMENTED_IMAGES_DIR_PATH + image
            aug_mask_file = AUGUMENTED_MASKS_DIR_PATH + image
            cv2.imwrite(image_file, transformed_image)
            cv2.imwrite(aug_mask_file, transformed_mask)
            for i in range(0, len(transformed_bboxes)):
                label_box = str(transformed_class_labels[i]) + " " + str(transformed_bboxes[i][0]) + " " + str(transformed_bboxes[i][1]) + " " + str(transformed_bboxes[i][2]) + " " + str(transformed_bboxes[i][3]) + "\n"
                label_boxes.append(label_box)
            label_file = AUGUMENTED_LABELED_IMAGES_DIR_PATH + image.split(".")[0] + ".txt"
            with open(label_file, "w") as ann:
                for lb in label_boxes:
                    ann.write(lb)

1/10505-VID795_G10_1_00d00h00m_0_1.png
2/10505-VID795_G10_1_00d00h00m_0_11.png
3/10505-VID795_G10_1_00d00h00m_0_8.png
4/10505-VID795_G10_1_00d00h00m_0_9.png
5/10505-VID795_G10_1_00d00h00m_10_4.png
6/10505-VID795_G10_1_00d00h00m_10_5.png
7/10505-VID795_G10_1_00d00h00m_11_4.png
8/10505-VID795_G10_1_00d00h00m_11_5.png
9/10505-VID795_G10_1_00d00h00m_1_8.png
10/10505-VID795_G10_1_00d00h00m_1_9.png
11/10505-VID795_G10_1_00d00h00m_2_0.png
12/10505-VID795_G10_1_00d00h00m_2_1.png
13/10505-VID795_G10_1_00d00h00m_3_0.png
14/10505-VID795_G10_1_00d00h00m_3_1.png
15/10505-VID795_G10_1_00d02h00m_0_0.png
16/10505-VID795_G10_1_00d02h00m_0_1.png
17/10505-VID795_G10_1_00d02h00m_0_10.png
18/10505-VID795_G10_1_00d02h00m_0_11.png
19/10505-VID795_G10_1_00d02h00m_0_2.png
20/10505-VID795_G10_1_00d02h00m_0_3.png
21/10505-VID795_G10_1_00d02h00m_0_4.png
22/10505-VID795_G10_1_00d02h00m_0_5.png
23/10505-VID795_G10_1_00d02h00m_0_6.png
24/10505-VID795_G10_1_00d02h00m_0_7.png
25/10505-VID795_G10_1_00d02h00m_0_8.png
26

In [7]:
#-----------------------------------------OLD--------------------------------------------
images = sorted(os.listdir(GREEN_PATH))
len_images = len(images)
curr_image_num = 1
LABELED_IMAGES_DIR_PATH = IMAGES_PATH + "Labeles_phase/"
AUGUMENTED_LABELED_IMAGES_DIR_PATH = IMAGES_PATH + "Labels_phase_aug/"
AUGUMENTED_IMAGES_DIR_PATH = IMAGES_PATH + "TSV_phase_aug/"
MASKS_DIR_PATH = IMAGES_PATH + "Masks_phase/"
AUGUMENTED_MASKS_DIR_PATH = IMAGES_PATH + "Masks_phase_aug/"

for image in images:
    print(str(curr_image_num) + "/" + str(len_images) + "-" + image)
    curr_image_num+=1
    base_image_path = PHASE_PATH + image
    annotation_path = LABELED_IMAGES_DIR_PATH + image.split(".")[0] + ".txt"
    mask_image_path = MASKS_DIR_PATH + image
    if os.path.exists(annotation_path):
        img_base = cv2.imread(base_image_path, cv2.IMREAD_UNCHANGED)
        img_mask = cv2.imread(mask_image_path, cv2.IMREAD_UNCHANGED)
        bboxes, class_labels = read_yolo_labels(annotation_path)
        for k in range(5):
            transformed = transform(image=img_base, bboxes=bboxes, class_labels=class_labels, phase_mask=img_mask)
            transformed_image = transformed['image']
            transformed_bboxes = transformed['bboxes']
            transformed_class_labels = transformed['class_labels']
            transformed_mask = transformed['phase_mask']

            if len(transformed_bboxes):
                print(k)
                label_boxes = []
                image_file = AUGUMENTED_IMAGES_DIR_PATH + image.split(".")[0] + "_" + str(k) + ".png"
                aug_mask_file = AUGUMENTED_MASKS_DIR_PATH + image.split(".")[0] + "_" + str(k) + ".png"
                #transformed_image = cv2.resize(transformed_image, (128,128))
                #transformed_mask = cv2.resize(transformed_mask, (128,128))
                cv2.imwrite(image_file, transformed_image)
                cv2.imwrite(aug_mask_file, transformed_mask)
                for i in range(0, len(transformed_bboxes)):
                    label_box = str(transformed_class_labels[i]) + " " + str(transformed_bboxes[i][0]) + " " + str(transformed_bboxes[i][1]) + " " + str(transformed_bboxes[i][2]) + " " + str(transformed_bboxes[i][3]) + "\n"
                    label_boxes.append(label_box)
                label_file = AUGUMENTED_LABELED_IMAGES_DIR_PATH + image.split(".")[0] + "_" + str(k) + ".txt"
                with open(label_file, "w") as ann:
                    for lb in label_boxes:
                        ann.write(lb)
    #break

1/312-VID746_C10_1_00d00h00m.png
2/312-VID746_C10_1_00d02h00m.png
4
3/312-VID746_C10_1_00d04h00m.png
0
1
3
4/312-VID746_C10_1_00d06h00m.png
3
5/312-VID746_C10_1_00d08h00m.png
1
2
6/312-VID746_C10_1_00d10h00m.png
7/312-VID746_C10_1_00d12h00m.png
8/312-VID746_C10_1_00d14h00m.png
9/312-VID746_C10_1_00d16h00m.png
10/312-VID746_C10_1_00d18h00m.png
11/312-VID746_C10_1_00d20h00m.png
0
12/312-VID746_C10_1_00d22h00m.png
13/312-VID746_C10_1_01d00h00m.png
14/312-VID746_C10_2_00d00h00m.png
15/312-VID746_C10_2_00d02h00m.png
16/312-VID746_C10_2_00d04h00m.png
17/312-VID746_C10_2_00d06h00m.png
18/312-VID746_C10_2_00d08h00m.png
19/312-VID746_C10_2_00d10h00m.png
0
20/312-VID746_C10_2_00d12h00m.png
0
21/312-VID746_C10_2_00d14h00m.png
22/312-VID746_C10_2_00d16h00m.png
2
23/312-VID746_C10_2_00d18h00m.png
2
4
24/312-VID746_C10_2_00d20h00m.png
25/312-VID746_C10_2_00d22h00m.png
26/312-VID746_C10_2_01d00h00m.png
27/312-VID746_C11_1_00d00h00m.png
0
1
28/312-VID746_C11_1_00d02h00m.png
2
4
29/312-VID746_C11_1_00d